<a href="https://colab.research.google.com/github/nrohrbach/PocDatenkatalogBafu/blob/main/PoC_Datenkatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test
Daten: http://www.bafu-daten.ch/wasser/restwasser/data/data/index/rawData_d.txt

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
Options = ['Abfall',
           'Altlasten',
           'Bildung, Forschung, Innovation',
           'Biodiversität',
           'Biotechnologie',
           'Boden',
           'Chemikalien',
           'Elektrosmog und Licht',
           'Ernährung, Wohnen, Mobilität',
           'Gesundheit',
           'Internationales',
           'Klima',
           'Landschaft',
           'Lärm',
           'Luft',
           'Naturgefahren',
           'Recht',
           'Störfallvorsorge',
           'Umweltverträglichkeitsprüfung',
           'Wald & Holz',
           'Wasser',
           'Wirtschaft und Konsum'
           ]

In [3]:
def map_options(option):
  mapping = {
      'Abfall': 'abfall',
      'Altlasten': 'altlasten',
      'Bildung, Forschung, Innovation': 'bildung',
      'Biodiversität': 'biodiversitat',
      'Biotechnologie' : 'biotechnologie',
      'Boden' : 'boden',
      'Chemikalien' : 'chemikalien',
      'Elektrosmog und Licht' : 'elektrosmog',
      'Ernährung, Wohnen, Mobilität'  : 'ernaehrung',
      'Gesundheit'  : 'gesundheit',
      'Internationales'  : 'internationales',
      'Klima'  : 'klima',
      'Landschaft'  : 'landschaft',
      'Lärm'  : 'laerm',
      'Luft'  : 'luft',
      'Naturgefahren'  : 'naturgefahren',
      'Recht'  : 'recht',
      'Störfallvorsorge'  : 'storfallvorsorge',
      'Umweltverträglichkeitsprüfung'  : 'umweltvertraeglichkeitpruefung',
      'Wald & Holz'  : 'wald',
      'Wasser'  : 'wasser',
      'Wirtschaft und Konsum'  : 'wirtschaft'
  }
  return mapping.get(option, None) # Return None if option is not found


In [22]:
# Funktion fragt alle Datensätze von opendata.swiss nach einem Keyword ab

def search_datasets(keyword):
    # Basis-URL der CKAN API
    base_url = "https://opendata.swiss/api/3/action/package_search"

    # Parameter für die Abfrage
    params = {
        "fq": f"organization:{'bundesamt-fur-umwelt-bafu'}",
        "q": f"keywords_de:{keyword}"
    }

    # Anfrage an die API senden
    response = requests.get(base_url, params=params)
    data = response.json()
    data = data["result"]["results"]

    # Daten aus Json auslesen
    description = [s['description']['de'] for s in data]
    title = [s['title']['de'] for s in data]
    url0 = [s['resources'][0]['url'] for s in data]
    title0 = [s['resources'][0]['title']['de'] for s in data]
    url1 = [s['resources'][1]['url'] for s in data]
    title1 = [s['resources'][1]['title']['de'] for s in data]

    # Daten als Dataframe speichern
    dict = {'title': title, 'description': description, 'Link': url0}
    dfOpendataSwiss = pd.DataFrame(dict)

    return dfOpendataSwiss

In [26]:
search_datasets('wald')

,title,description,Link
0,LFI: Themen nach NaiS-Vegetationshöhenstufen (...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=%23pragm...
1,LFI: Themen nach Schadenausmass (10 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
2,LFI: Themen nach Eigentum (2 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
3,LFI: Themen nach Stücklänge liegender toter Bä...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
4,LFI: Themen nach Schafthöhe gebrochener Dürrst...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=%23pragm...
5,LFI: Themen nach NaiS-Vegetationshöhenstufen (...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
6,LFI: Themen nach Standortgüte,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
7,LFI: Themen nach Baumart (56 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
8,LFI: Themen nach eingeführte Baumart (baumförm...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
9,LFI: Themen nach Baumzustand (stehend/liegend),Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...


In [24]:
def filter_df_by_thema(thema):
    url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

    try:
      soup = BeautifulSoup(response.content, "html.parser")
      table_body = soup.find("table", attrs={"class": "table"})

      if not table_body:
        print("Table not found on the webpage")
        return None

      rows = table_body.find_all("tr")
      data = []
      for row in rows:
          cells = row.find_all("td")
          if len(cells) >= 2:
              thema_cell = cells[0].text.strip()
              indikatorname = cells[1].text.strip()
              link_element = cells[1].find('a')
              href = link_element.get('href') if link_element else None
              data.append({"Thema": thema_cell, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch" + href if href else None})

      df = pd.DataFrame(data)
    except Exception as e:
      print(f"An error occurred while processing the webpage: {e}")
      return None

    # Filter the DataFrame using .isin() for multiple Thema values if needed
    if isinstance(thema, list):
      filtered_df = df[df["Thema"].isin(thema)]
    else:
      filtered_df = df[df["Thema"] == thema]

    return filtered_df


In [25]:
filter_df_by_thema('Abfall')

,Thema,Indikatorname,Link
0,Abfall,Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
1,Abfall,Energieeffizienz bei der Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
2,Abfall,Verwertete Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
119,Abfall,Auf Deponien abgelagerte Abfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
120,Abfall,Begleitscheine Sonderabfalltransporte,https://www.bafu.admin.ch/bafu/de/home/themen/...
121,Abfall,Energieproduktion aus Abfällen,https://www.bafu.admin.ch/bafu/de/home/themen/...
122,Abfall,Exportierte Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
123,Abfall,Grenzüberschreitender Verkehr mit Abfällen,https://www.bafu.admin.ch/bafu/de/home/themen/...
124,Abfall,Importierte Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
125,Abfall,In der Schweiz anfallende Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
